In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import functools

import warnings
warnings.filterwarnings('ignore')

: 

In [ ]:
movies = pd.read_csv('movies_metadata.csv')
movies.head(3)

In [ ]:
movies.shape

In [ ]:
movies['genres'].head(3)

In [ ]:
[i['name'] for i in literal_eval(movies['genres'].iloc[0])]

In [ ]:
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies['genres'].head(3)

In [ ]:
available_genres = functools.reduce(lambda x, y: set(x).union(set(y)), movies['genres'].tolist())
available_genres

In [ ]:
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies['year'] = movies['release_date'].dt.year
movies['year'].iloc[0]

In [ ]:
movies[['vote_average', 'vote_count']].dtypes

In [ ]:
movies[['vote_average', 'vote_count']].isna().sum()

Для создания неперсо
нализированной рекомендации используем не все данные, только 'title', 'year', 'vote_count', 'vote_average', 'genres'

In [ ]:
def proc_dataset(df):
  dataset = df.dropna(subset=['vote_average', 'vote_count'], axis=0)
  dataset = dataset[['title', 'year', 'vote_count', 'vote_average', 'genres']]
  return dataset

In [ ]:
nonpers_df = proc_dataset(movies)
nonpers_df.head(3)

Для того, чтобы лучше подобрать рекомендации для новых пользователей необходимо правильно задать формулу.


Есть несколько идей:

1) Можем предлагать фильмы с самым высоким значением 'vote_average'.

2) Предлагать фильмы с самым высоким 'vote_average', но учитывать также 'vote_count'.

3) Учитывать среднее значение 'vote_average' и текущее значение 'vote_average'. Взвешенное среднее.

4) К 3) пункту добавить также интересующие пользователя жанры.

5) Можно добавить к 4) пункту фильтрацию по дате релиза фильма. Например, если пользователь не любит старые фильмы, которые выпущены до 2000г. (Учитывать интересы пользователя)

Реализуем 1-ю идею.

In [ ]:
def first_idea(df):
  return df['vote_average']

def recomend_first(movies, n):
  df = movies.copy()
  df['formula'] = df.apply(lambda x: first_idea(x), axis=1)
  df.sort_values('formula', ascending=False, inplace=True)
  return df.head(n).reset_index(drop=True)

In [ ]:
recomend_first(nonpers_df, 10)

Рейтинг у каждого фильма высокий, но можем заметить, что это оценки одного пользователя. Скорее всего, вряд ли эти фильмы заинтересуют новых пользователей, так как это предпочтения одного пользователя, и они могут быть специфическими.

По этому необходимо учитывать количество 'vote_count'.

Реализуем 2-ю идею.

In [ ]:
def second_idea(df):
  R = df['vote_average']
  q = df['vote_count']
  return R * q

def recomend_second(movies, n):
  df = movies.copy()
  df['formula'] = df.apply(lambda x: second_idea(x), axis=1)
  df.sort_values('formula', ascending=False, inplace=True)
  return df.head(10).reset_index(drop=True)

In [ ]:
recomend_second(nonpers_df, 10)

Это идея неплоха, но не все фильмы, которые получили большее количество голосов, являются лучшими. В этом случае мы не учитываем фильмы, у которых высокий рейтинг, но их оценило не так много людей. Это может происходить в тех случаях, когда фильм только вышел и не успел набрать свою аудиторию или у фильма была слабая реклама,и о нем знает небольшое количество людей. То есть небольшое количество голосов не значит, что фильм плохой. Следовательно, их тоже необходимо учитывать.

Реализуем 3-ю идею.

In [ ]:
def third_idea(df,m,q):
  R = df['vote_average']
  p = df['vote_count']
  return (p / (p + q)) * R + (q / (q + p)) * m

def recomend_third(movies, n):
  df = movies.copy()
  m = np.mean(df['vote_average'])
  q = np.quantile(df['vote_count'], 0.95)

  df['formula'] = df.apply(lambda x: third_idea(x, m, q), axis=1)
  df.sort_values('formula', ascending=False, inplace=True)
  return df.head(n).reset_index(drop=True)

In [ ]:
recomend_third(nonpers_df, 10)

Можем заметить, что у некоторых фильмов количество голосов меньше, но при этом у них хороший рейтинг. Для этой идеии мы использовали взвешенное среднее, то есть каждый член имел свой вес(учитывалось важность отдельных компонент).


В нашем случае,чем больше голосов у фильма, тем больше учитывается его собственный рейтинг. Если наоборот, у фильма мало голосов, то вторая часть формулы помогает "поднять" рейтинг менее популярных фильмов.

Реализуем 4-ю идею.

Теперь хотелось бы учитывать пожелания пользователя по жанрам. То есть можно в начале узнать, может есть какие-то предпочтеня.

Здесь могут возникнуть 2 ситуации:
1) Выбрав жанры, юзер хочет посмотреть фильм, подходящий хотя бы по одному из выбранных жанров. То есть он не должен удолетворять всем запрошенным параметрам.

2) Или наоборот, пользователь хочет, чтобы фильм подходил по всем выбранным жанрам.

1 ситуация:

In [ ]:
def recomend_4th_s1(movies, wishes, n):
  df = movies.copy()
  m = np.mean(df['vote_average'])
  q = np.quantile(df['vote_count'], 0.95)

  df['formula'] = df.apply(lambda x: third_idea(x, m, q), axis=1)
  df.sort_values('formula', ascending=False, inplace=True)

  df['certain_genres'] = [any(y in x for y in wishes) for x in df['genres']]
  df = df[df['certain_genres'] == True]
  df.drop('certain_genres', axis=1, inplace=True)
  return df.head(n).reset_index(drop=True)

In [ ]:
recomend_4th_s1(nonpers_df, ['Comedy', 'Fantasy'], 10)

2 ситуация:

In [ ]:
def recomend_4th_s2(movies, wishes, n):
  df = movies.copy()
  m = np.mean(df['vote_average'])
  q = np.quantile(df['vote_count'], 0.95)

  df['formula'] = df.apply(lambda x: third_idea(x, m, q), axis=1)
  df.sort_values('formula', ascending=False, inplace=True)

  df['certain_genres'] = [all(y in x for y in wishes) for x in df['genres']]
  df = df[df['certain_genres'] == True]
  df.drop('certain_genres', axis=1, inplace=True)
  return df.head(n).reset_index(drop=True)

In [ ]:
recomend_4th_s2(nonpers_df, ['Comedy', 'Fantasy'], 10)

Как видим, результаты разные в обоих случаях, поэтому стоит учточнить в начале у юзера этот пункт.

Реализуем 5-ю идею.

In [ ]:
def recomend_5th(movies, wishes_genres, wishes_year, n):
  df = movies.copy()
  m = np.mean(df['vote_average'])
  q = np.quantile(df['vote_count'], 0.95)

  df['formula'] = df.apply(lambda x: third_idea(x, m, q), axis=1)
  df.sort_values('formula', ascending=False, inplace=True)

  df['certain_genres'] = [any(y in x for y in wishes_genres) for x in df['genres']]
  df = df[df['certain_genres'] == True]
  df.drop('certain_genres', axis=1, inplace=True)

  if wishes_year.startswith('С'):
    df = df[df['year'] >= int(wishes_year.split()[-1])]

  elif wishes_year.startswith('До'):
    df = df[df['year'] <= int(wishes_year.split()[-1])]

  return df.head(n).reset_index(drop=True)


In [ ]:
recomend_5th(nonpers_df, ['Comedy', 'Fantasy'], 'С 2010', 10)

In [ ]:
recomend_5th(nonpers_df, ['Animation'], 'До 2010', 10)

Вот и реализация popularity-based рекомендаций. Они основаны на том, что юзеру понравится то, что и всем остальным.

Это можно использовать, если пользователь новый. Т.е. "холодный старт", когда нам ничего неизвестно о новом пользователе.